<a href="https://colab.research.google.com/github/svplaksin/simulative-python/blob/main/%D0%9A%D0%B5%D0%B9%D1%81_%D0%9B%D0%BE%D0%B3%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [ ]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[400:550]:
    print(line)

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [ ]:

def count_success_and_failure(file_path):
  with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()
    errors = [line for line in lines if line.startswith('ERROR')]
    success = [line for line in lines if "Обновляем подписку пользователю id:" in line]
    return len(success) - len(errors), len(errors)


count_success_and_failure('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [ ]:
import re
import statistics
from datetime import datetime
from collections import defaultdict
from math import floor

def auto_renewal_sub(log_file_path):

    with open(log_file_path, 'r') as f:
      lines = f.readlines()

    date_values = {}
    pattern = r'Cегодня (\d{4}-\d{2}-\d{2}).*?подписки: (\d+)'

    for line in lines:
      match = re.search(pattern, line)
      if match:
        date_str, count_str = match.groups()
        date = datetime.strptime(date_str, '%Y-%m-%d').date()
        count = int(count_str)

        if date not in date_values or count > date_values[date]:
          date_values[date] = count
    values = list(date_values.values())
    # Рассчитываем скольящие
    moving_avg = []
    moving_median = []

    for i in range(len(values)):
      window = values[:i+1]
      mean_val = round(statistics.mean(window), 2)
      moving_avg.append(float(mean_val))

      median_val = statistics.median(window)
      if median_val.is_integer():
        moving_median.append(int(median_val))
      else:
        moving_median.append(floor(median_val))
    # Записываем результаты
    with open('auto_renewal_sub.txt', 'w') as f:
      f.write(f"Среднее: {moving_avg}\n")
      f.write(f"Медиана: {moving_median}")

auto_renewal_sub('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

In [ ]:
# print(user_answer)
# print(correct_answer)
print(user_answer.columns)
print('--')
print(correct_answer.columns)

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [ ]:

def sub_renewal_by_day(file_path):

    day_counts = defaultdict(int)
    days_map = {
        0: "Понедельник",
        1: "Вторник",
        2: "Среда",
        3: "Четверг",
        4: "Пятница",
        5: "Суббота",
        6: "Воскресенье"
    }

    with open(file_path, 'r') as f:
      for line in f:
        if 'Обновляем подписку пользователю' in line:
          match = re.search(r'(\d{4}-\d{2}-\d{2})', line)
          date_obj = datetime.strptime(match.group(1), '%Y-%m-%d')
          weekday = date_obj.weekday()
          day_counts[weekday] += 1

        elif line.startswith("ERROR"):
          match = re.search(r'(\d{4}-\d{2}-\d{2})', line)
          date_obj = datetime.strptime(match.group(1), '%Y-%m-%d')
          weekday = date_obj.weekday()
          day_counts[weekday] -= 1

    # Записываем отчет
    with open('weekdays.txt', 'w') as f:
      f.write('Количество обновлений подписки по дням недели:\n')
      for i in range(7):
        f.write(f'{days_map[i]}: {day_counts.get(i, 0)}\n')
sub_renewal_by_day('auto_purchase.log')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')